# LIBRARY

In [1]:
 pip install surprise

In [2]:
import numpy as np 
import pandas as pd 
import time

import os 
%matplotlib inline
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

pd.options.display.max_columns=None

import warnings; warnings.simplefilter('ignore')

# LOAD DATASET

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
begin = time.time()
data1 = pd.read_csv("/content/drive/MyDrive/BIGDATA/DOAN/NEWDATASET/movies_metadata.csv")

In [5]:
data1.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# DATA PREPROCESSING

In [6]:
# Xác định tổng số giá trị null trong dữ liệu
data1.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

## DATA CLEANING

In [7]:
# Data cleaning
# fillna -> [] đối với cac json trong data, sau đó đánh giá dữ liệu có phải các kiểu python và parse json để lấy data ra
# isinstance(x, list) -> xác định 1 data có phải là kiểu type đó hay không
data1['genres'] = data1['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else []) # Kiểm tra 
data1['production_companies']= data1['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
data1['production_countries'] = data1['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
data1['spoken_languages'] = data1['spoken_languages'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
# chuyển dổi string date thành các object sau đó split theo "-" - errors='coerce' => nếu string ko phải time -> NaT
data1['year'] = pd.to_datetime(data1['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [8]:
data1.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995


## INTRODUCTION - DEMOGRAPHIC FILTERING - WEIGHT RANK (WR)

In [9]:
# Sử dụng Xếp hạng IMDB để đưa ra Bảng xếp hạng các bộ phim hàng đầu. 
# Sử dụng công thức xếp hạng có trọng số của IMDB để xây dựng biểu đồ của mình. Về mặt toán học, nó được biểu diễn như sau:
# Weighted Rank (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
# https://www.imdb.com/
# R = lượt vote trung bình trong 1 bộ phim (mean) = (vote_average)
# v = Số vote trong 1 bộ phim = (vote_count)
# m = Số phiếu tối thiểu để có thể xếp hạng
# C = lượt vote trung bình trên toàn bộ các bộ phim

# xác định m => để một bộ phim có mặt trong bảng xếp hạng, nó phải có số phiếu bình chọn >= 95% các bộ phim trong danh sách.
# tức là sẽ có (100 -95)% số trường hợp quan sát thấy thấp hơn m và sẽ có nhiều nhất 95% số trường hợp lớn hơn m

In [10]:
vote_counts = data1[data1['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = data1[data1['vote_average'].notnull()]['vote_average'].astype('int')
m = vote_counts.quantile(0.95)
def weighted_rating(x):
  C = vote_averages.mean()
  # Dùng phân vị 95 để tìm ra được m
  v = x['vote_count']
  R = x['vote_average']
  return (v/(v+m) * R) + (m/(m+v) * C)

# Lấy ra bộ phân data có vote_count >= m vói 6 features 'title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres'
qualified = data1[(data1['vote_count'] >= m) & (data1['vote_count'].notnull()) & (data1['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
# Chuyển type data
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
print('Cấu trúc mới của hệ dữ liệu là = ',qualified.shape)
# 6 feature và 1819 dòng

qualified['wr'] = qualified.apply(weighted_rating, axis=1)
# sort theo wr từ cao -> thấp
qualified = qualified.sort_values('wr', ascending=False)

Cấu trúc mới của hệ dữ liệu là =  (2274, 6)


### INTRODUCTION - BEST RANK

In [11]:
# top 15 movies được xếp hạng cao nhất
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


### INTRODUCTION - GENRES RANK

In [12]:
# Tổng hợp tất cả các tập data dựa trên thể loại
s = data1.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = data1.drop('genres', axis=1).join(s)
gen_md.head(8)

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,False,7.7,5415.0,1995,Family
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Adventure
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Fantasy
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,Family
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Romance
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,Comedy


In [13]:
def rank_genre(genre, percentile=0.90):
  # Lấy df theo genre
  df = gen_md[gen_md['genre'] == genre]
  vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
  vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')

  C = vote_averages.mean()
  m = vote_counts.quantile(percentile)
  
  # Láy dataframe có chỉ số lớn hơn m và không được null vote_count và vote_average
  qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
  # đổi kiểu int
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')
  
  # Công thức tính WR
  qualified['weight_rank'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
  qualified = qualified.sort_values('weight_rank', ascending=False)
  
  return qualified

In [14]:
rank_genre('Horror').head(15)

,title,year,vote_count,vote_average,popularity,weight_rank
1213,The Shining,1980,3890,8,19.6116,7.837897
1176,Psycho,1960,2405,8,36.8263,7.745768
1171,Alien,1979,4564,7,23.3774,6.904360
41492,Split,2016,4461,7,28.920839,6.902250
14236,Zombieland,2009,3655,7,11.063,6.881855
1158,Aliens,1986,3282,7,21.7612,6.869227
21276,The Conjuring,2013,3169,7,14.9017,6.864851
42169,Get Out,2017,2978,7,36.894806,6.856749
1338,Jaws,1975,2628,7,19.7261,6.839069
8147,Shaun of the Dead,2004,2479,7,14.9029,6.830145


In [15]:
rank_genre('Comedy').head(15)

,title,year,vote_count,vote_average,popularity,weight_rank
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.122862
351,Forrest Gump,1994,8147,8,48.3072,7.933642
1225,Back to the Future,1985,6239,8,25.7785,7.914000
18465,The Intouchables,2011,5410,8,16.0869,7.901308
22841,The Grand Budapest Hotel,2014,4644,8,14.442,7.885723
2211,Life Is Beautiful,1997,3643,8,39.395,7.856012
732,Dr. Strangelove or: How I Learned to Stop Worr...,1964,1472,8,9.80398,7.669830
3342,Modern Times,1936,881,8,8.15956,7.490482
883,Some Like It Hot,1959,835,8,11.8451,7.467989
1236,The Great Dictator,1940,756,8,9.24175,7.424345


## MERGE TAGLINE & OVERVIEW

In [16]:
data1 = data1.drop([19730, 29503, 35587])
data1['id'] = data1['id'].astype('int')
# Lấy 1/4 data trong movies vì bị tràn RAM
links_small = pd.read_csv('/content/drive/MyDrive/BIGDATA/DOAN/NEWDATASET/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
smd = data1[data1['id'].isin(links_small)]
smd['tagline'] = smd['tagline'].fillna('')
smd['overview'] = smd['overview'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')
len(smd)

9099

In [17]:
smd.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,description
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,,Toy Story,False,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...


# CONTENT BASED FILTERING

## TFIDF VECTORIZER

In [18]:
print(smd['description'])
# dùng tfidf chuyền chuỗi thành các vector - loại bỏ các stop word, phân thành các ngram từ, và giới hạn tần suất xuất hiện thấp nhất được đề ra
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

0        Led by Woody, Andy's toys live happily in his ...
1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
                               ...                        
40224    From the mind behind Evangelion comes a hit la...
40503    The band stormed Europe in 1963, and, in 1964,...
44821    When Molly Hale's sadness of her father's disa...
44826    All your favorite Pokémon characters are back,...
45265    While holidaying in the French Alps, a Swedish...
Name: description, Length: 9099, dtype: object


In [19]:
tfidf_matrix.shape

(9099, 268124)

## COSINE SIMILARITY

In [20]:
# lập ma trận đối để tính cosine với toàn độ data với nhau
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:
smd = smd.reset_index()
titles = smd['title']
# tạo indicate với index
indices = pd.Series(smd.index, index=smd['title'])
print(indices)

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
Pokémon: Spell of the Unknown                         9096
Pokémon 4Ever: Celebi - Voice of the Forest           9097
Force Majeure                                         9098
Length: 9099, dtype: int64


In [22]:
def get_recommendations(title):
    movies = np.array(smd[smd['title'].str.contains(title)].title.index)
    print(f"Có {len(movies)} được tìm thấy")
    for i,m in enumerate(movies):
      print(f"{i+1} --- {smd.loc[m]['title']}")

    id_select = int(input("Chọn một bộ phim: "))
    movie = movies[id_select - 1]
    print(f"Phim được chọn: {smd.loc[movie].title}\n----------------------------\n Phim được đề xuất")
    idx = indices[smd.loc[movie].title]
    # Tính cosine similarity
    sim_scores = list(enumerate(cosine_sim[idx]))
    # sort
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # bỏ cái đầu vì trùng
    sim_scores = sim_scores[1:11]
    movie_indices = [[i[1],smd.loc[i[0]].title] for i in sim_scores]
    result_df = pd.DataFrame(movie_indices,columns=['Similarity','Movies'])
    end = time.time()
    result_time = end - begin
    print("Time :" ,result_time)
    return result_df

## RESULT - MOVIES RECOMMENDATION - HARRY POTTER

In [23]:
get_recommendations('Harry Potter')

Có 8 được tìm thấy
1 --- Harry Potter and the Philosopher's Stone
2 --- Harry Potter and the Chamber of Secrets
3 --- Harry Potter and the Prisoner of Azkaban
4 --- Harry Potter and the Goblet of Fire
5 --- Harry Potter and the Order of the Phoenix
6 --- Harry Potter and the Half-Blood Prince
7 --- Harry Potter and the Deathly Hallows: Part 1
8 --- Harry Potter and the Deathly Hallows: Part 2
Chọn một bộ phim: 7
Phim được chọn: Harry Potter and the Deathly Hallows: Part 1
----------------------------
 Phim được đề xuất
Time : 49.606985569000244


,Similarity,Movies
0,0.210049,Harry Potter and the Goblet of Fire
1,0.182423,Harry Potter and the Deathly Hallows: Part 2
2,0.139190,Harry Potter and the Chamber of Secrets
3,0.139183,Harry Potter and the Prisoner of Azkaban
4,0.115248,Harry Potter and the Half-Blood Prince
5,0.114324,Harry Potter and the Order of the Phoenix
6,0.063206,Harry and Tonto
7,0.053839,Miracle Mile
8,0.050756,K-19: The Widowmaker
9,0.050170,The Last Witch Hunter
